In [ ]:
# Install FastAPI, Uvicorn, nest_asyncio (if not already installed)
# !pip install fastapi uvicorn nest_asyncio requests

import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel
from uvicorn import Config, Server
import asyncio
import socket
from threading import Thread
import requests

# Enable asyncio to work within Jupyter Notebook
nest_asyncio.apply()

# Create FastAPI instance
app = FastAPI()

# Define a Pydantic model for the input message
class Message(BaseModel):
    message: str

# Define an endpoint for inference
@app.post("/inference")
async def inference(message: Message):
    """
    Endpoint that takes a message and returns a response with "Hello world" and the message.
    """
    # Forward the request to the code engine tunnel
    try:
        # Make a POST request to the Code Engine tunnel's /ncic endpoint to create a tunnel
        tunnel_response = requests.post("http://localhost:5000/ncic", json={'port': 8000})  # Assuming your FastAPI app runs on port 8000
        tunnel_response.raise_for_status()
        public_url = tunnel_response.json()['public_url']

        # Now, use the public_url to make the actual request
        response = requests.post(f"{public_url}/inference", json={"message": message.message})
        response.raise_for_status()  # Raise an exception for bad status codes
        code_engine_response = response.json()
        return {"response": f"Hello world {message.message} from IBM CIC, Code Engine says: {code_engine_response['message']}", "data": code_engine_response.get('data')}
    except requests.exceptions.RequestException as e:
        return {"error": f"Error communicating with Code Engine tunnel: {e}"}


# Function to find an available port
def find_free_port(default_port=8000):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.bind(("0.0.0.0", default_port))
            return default_port
        except OSError:
            # Find an available port if the default is in use
            s.bind(("0.0.0.0", 0))
            return s.getsockname()[1]

# Function to start the Uvicorn server
def start_uvicorn(app, port):
    """
    Start the Uvicorn server in the current thread.
    """
    config = Config(app=app, host="0.0.0.0", port=port, log_level="info")
    server = Server(config)
    asyncio.run(server.serve())

# --- Custom ncic implementation using the Code Engine tunnel ---

def ncic_custom(port, authtoken=None, region=None):
    """
    Simulates ncic functionality using the Code Engine tunnel.
    """
    try:
        # Request a new tunnel from the Code Engine app
        response = requests.post(f"http://localhost:5000/ncic", json={'port': port})
        response.raise_for_status()
        return response.json()['public_url']
    except requests.exceptions.RequestException as e:
        print(f"Error creating tunnel: {e}")
        return None


# --- End of custom ncic implementation ---

# Function to run the server with the custom ncic
def run_server_with_ncic(app, port, authtoken=None, duration=60):
    """
    Run the FastAPI server with the custom ncic implementation.
    """
    # Use the custom ncic function to get the tunnel URL
    public_url = ncic_custom(port, authtoken)  

    print(f"Public URL for the API: {public_url}/inference")
    print(f"To test the API using curl, run the following command:")
    print(f"curl -X POST {public_url}/inference -H 'Content-Type: application/json' -d '{{\"message\": \"Your test message\"}}'")
    print(f"Or paste the following link into your web browser:")
    print(f"{public_url}/inference")

    # Start Uvicorn server in a separate thread
    uvicorn_thread = Thread(target=start_uvicorn, args=(app, port), daemon=True)
    uvicorn_thread.start()

    try:
        # Wait for the specified duration
        print(f"The server will shut down in {duration} seconds...")
        asyncio.get_event_loop().run_until_complete(asyncio.sleep(duration))
    except KeyboardInterrupt:
        print("Shutdown requested manually.")
    finally:
        # No need to disconnect from ncic here as we're using our custom implementation
        print("Shutting down the server...")
        uvicorn_thread.join(timeout=1)

# Main entry point
if __name__ == "__main__":
    print("Starting the API with custom ncic...")

    # Find an available port
    port = find_free_port(8000)

    # Run the server with the custom ncic for 1 minute
    run_server_with_ncic(app, port, duration=60)